# Training notebook

Here we tune parameters of models. From the 7 available seasons, we use first 4 for training and the rest 3 go in testing. There is no fitting on data as such and so we just performance over 4 training seasons to see which parameters work best.

In [28]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
np.random.seed(1234)

import sys
sys.path.append("../src")
from utils import data as udata
from utils import dists as udists
from utils import misc as u
from truth import mask_truths
from predictors import make_predictor
import losses
from pymmwr import Epiweek
from tqdm import tqdm, trange
import ledge.merge as merge
import ledge.update as update
from functools import partial

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
EXP_DIR = "../data/processed/cdc-flusight-ensemble/"
OUTPUT_DIR = "../models/cdc-flusight-ensemble/"
TARGET = "1-ahead"
MAX_LAG = 29
REGIONS = ["nat", *[f"hhs{i}" for i in range(1, 11)]]
REGIONS = ["nat"] # Speed up!
TRAINING_SEASONS = list(range(2010, 2014))
LOSS_FN = losses.ploss

In [30]:
components = [udata.Component(EXP_DIR, m) for m in udata.available_models(EXP_DIR)]
ad = udata.ActualData(EXP_DIR)

# Evaluation

In [31]:
def evaluate(predictor, loss_fn=losses.ploss):
    """
    Evaluate the predictor over all training seasons and regions and return mean score
    """
    
    first_losses = []
    final_losses = []
    
    with tqdm(total=len(TRAINING_SEASONS) * len(REGIONS)) as pbar:
        for season in TRAINING_SEASONS:
            for region in REGIONS:
                truths = [ad.get(TARGET, region, season, lag=l) for l in range(MAX_LAG + 1)]
                c_preds = [cmp.get(TARGET, region, season) for cmp in components]
            
                first_truth = merge.zero(truths)
                final_truth = merge.latest(truths)
                pred = predictor(truths, c_preds)
                first_losses.append(float(loss_fn(pred, first_truth).mean()))
                final_losses.append(float(loss_fn(pred, final_truth).mean()))
                pbar.update()

    return {
        "first_loss": np.mean(first_losses),
        "final_loss": np.mean(final_losses)
    }

In [26]:
update_fn = update.ftl
merge_fn = merge.zero
predictor = make_predictor(LOSS_FN, merge_fn, update_fn)

# Tuning

In [34]:
# Merge strategies
merge_st = [("zero", merge.zero), ("latest", merge.latest)]

## Follow the leader

In [35]:
for merge_id, merge_fn in merge_st:
    losses = evaluate(make_predictor(LOSS_FN, merge_fn, update.ftl), LOSS_FN)
    print(f"{merge_id}, {losses['first_loss'], losses['final_loss']}")

  0%|          | 0/4 [00:00<?, ?it/s]

zero, (0.8572893443772136, 0.9007682099397212)


100%|██████████| 4/4 [00:20<00:00,  5.08s/it]

latest, (0.8558966113027653, 0.8934405764400863)


## MW

In [36]:
for merge_id, merge_fn in merge_st:
    for eta in np.linspace(0.5, 1.0, 11):
        update_fn = partial(update.mw, eta=eta)
        losses = evaluate(make_predictor(LOSS_FN, merge_fn, update_fn), LOSS_FN)
        print(f"{merge_id}, {eta}: {losses['first_loss'], losses['final_loss']}")

  0%|          | 0/4 [00:00<?, ?it/s]

zero, 0.5: (0.8800236126573013, 0.8978411657567614)


  0%|          | 0/4 [00:00<?, ?it/s]

zero, 0.55: (0.8777744189700727, 0.8970100292176808)


  0%|          | 0/4 [00:00<?, ?it/s]

...

latest, 0.9: (0.8672534561248718, 0.8901393386791897)


  0%|          | 0/4 [00:00<?, ?it/s]

latest, 0.95: (0.8662375708334357, 0.8903008346009428)


100%|██████████| 4/4 [00:21<00:00,  5.32s/it]

latest, 1.0: (0.8697122091091656, 0.8942047744704603)


## Hedge

In [39]:
for merge_id, merge_fn in merge_st:
    for eta in np.linspace(0.5, 5.0, 11):
        update_fn = partial(update.hedge, eta=eta)
        losses = evaluate(make_predictor(LOSS_FN, merge_fn, update_fn), LOSS_FN)
        print(f"{merge_id}, {eta}: {losses['first_loss'], losses['final_loss']}")

  0%|          | 0/4 [00:00<?, ?it/s]

zero, 0.5: (0.8866194723930647, 0.9007701611618466)


  0%|          | 0/4 [00:00<?, ?it/s]

zero, 0.95: (0.8786988854604574, 0.8977764475960712)


  0%|          | 0/4 [00:00<?, ?it/s]

...

latest, 4.1: (0.864476030473778, 0.8911387987320996)


  0%|          | 0/4 [00:00<?, ?it/s]

latest, 4.55: (0.8636649135021445, 0.8909539240942543)


100%|██████████| 4/4 [00:21<00:00,  5.39s/it]

latest, 5.0: (0.86302364913953, 0.890812073202538)
